# Classifier NNs

## NNs summary
- NNs components
    - Dense
    - BatchNormalization
    - DropOut
    - SoftMax

- Optimizer
    - Adam

- Loss
    - BinaryCrossEntropy

- Metric
    - acc

<img src="figures/NNs_classifier.png" style="float: left; width: 550px;"/>  

In [1]:
import pickle
import pprint
from sklearn.model_selection import train_test_split
from os import path

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
test_cases = [
    'pglib_opf_case24_ieee_rts.pickle', 
    'pglib_opf_case73_ieee_rts.pickle',
    'pglib_opf_case162_ieee_dtc.pickle',
    'pglib_opf_case300_ieee.pickle',
]

In [3]:
# choose a dataset
case_idx = 0
file_dir = path.join('./datasets/', test_cases[case_idx])

In [4]:
infile = open(file_dir,'rb')
dataset = pickle.load(infile)
infile.close()

In [5]:
# train & test
x_train, x_test, y_train, y_test = train_test_split(dataset['x'],
                                                    dataset['y'],
                                                    test_size=0.2,
                                                    random_state=19)
# train & validation
x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                  y_train,
                                                  test_size=0.2,
                                                  random_state=19)

In [6]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

y_train = y_train.astype('float32')
y_val = y_val.astype('float32')
y_test = y_test.astype('float32')

In [7]:
print((x_train.shape, y_train.shape))
print((x_val.shape, y_val.shape))
print((x_test.shape, y_test.shape))

((32000, 24), (32000, 82))
((8000, 24), (8000, 82))
((10000, 24), (10000, 82))


In [8]:
inputs = keras.Input(shape=(x_train.shape[1], ), name='uncertainty_realization')

x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.BatchNormalization(name='bn_1')(x)
x = layers.Dropout(rate=0.2, name='dropout_1')(x)

x = layers.Dense(128, activation='relu', name='dense_2')(x)
x = layers.BatchNormalization(name='bn_2')(x)
x = layers.Dropout(rate=0.2, name='dropout_2')(x)

x = layers.Dense(64, activation='relu', name='dense_3')(x)
x = layers.BatchNormalization(name='bn_3')(x)
x = layers.Dropout(rate=0.2, name='dropout_3')(x
                                              )
outputs = layers.Dense(y_train.shape[1], name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [9]:
model.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.BinaryCrossentropy(),
    # List of metrics to monitor
    metrics=['acc'])

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
uncertainty_realization (Inp [(None, 24)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                1600      
_________________________________________________________________
bn_1 (BatchNormalization)    (None, 64)                256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
bn_2 (BatchNormalization)    (None, 128)               512       
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0     

In [11]:
print('> Fit model on training data')
history = model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=8,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val))

> Fit model on training data
Train on 32000 samples, validate on 8000 samples
Epoch 1/8
32000/32000 [==============================] - 4s 116us/sample - loss: 2.4356 - acc: 0.7435 - val_loss: 1.1520 - val_acc: 0.8913
Epoch 2/8
32000/32000 [==============================] - 2s 74us/sample - loss: 1.1958 - acc: 0.8819 - val_loss: 0.7545 - val_acc: 0.9366
Epoch 3/8
32000/32000 [==============================] - 3s 79us/sample - loss: 0.6766 - acc: 0.9332 - val_loss: 0.3428 - val_acc: 0.9710
Epoch 4/8
32000/32000 [==============================] - 3s 84us/sample - loss: 0.2747 - acc: 0.9693 - val_loss: 0.1125 - val_acc: 0.9911
Epoch 5/8
32000/32000 [==============================] - 3s 94us/sample - loss: 0.1037 - acc: 0.9882 - val_loss: 0.0404 - val_acc: 0.9968
Epoch 6/8
32000/32000 [==============================] - 3s 81us/sample - loss: 0.0339 - acc: 0.9957 - val_loss: 0.0018 - val_acc: 0.9998
Epoch 7/8
32000/32000 [==============================] - 3s 84us/sample - loss: 0.0096 - acc:

In [12]:
pp = pprint.PrettyPrinter(indent=4)
print('> History dict:')
pp.pprint(history.history)

> History dict:
{   'acc': [   0.74351764,
               0.8819245,
               0.9332394,
               0.9692562,
               0.98819107,
               0.995652,
               0.9981577,
               0.9994129],
    'loss': [   2.435572667121887,
                1.1958334478735924,
                0.6766350555419922,
                0.27468075984716417,
                0.10374657178111374,
                0.033875535631901585,
                0.009621334598188695,
                0.0033639278715581894],
    'val_acc': [   0.8913385,
                   0.93656534,
                   0.97099066,
                   0.9911174,
                   0.9967573,
                   0.9997711,
                   1.0,
                   1.0],
    'val_loss': [   1.151975214958191,
                    0.7545073247551918,
                    0.3428191515803337,
                    0.11253948998078704,
                    0.04039800438261591,
                    0.0017848335202834279,
  